In [143]:
import dash
from dash import dcc, html
import more_itertools
import pandas as pd
from dash.dependencies import Input,Output
import plotly.express as px
import plotly.graph_objs as go

# Load the data
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# List of years
year_list = [i for i in range(1980, 2024, 1)]

# Initialize Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1('Automobile Sales Statistics Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
    
    html.Div([
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown_statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
            ],
            value='Yearly Statistics',  # Set default value to match one of the options
            placeholder='Select a report type.'
        )
    ]),
    
    html.Div([
        html.Label("Select Year:"),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': str(i), 'value': i} for i in year_list],
            value=2020  # Set default year value
        )
    ]),
    
    # Division for output display (chart grid)
    html.Div(id='output-container', className='chart-grid', style={'display': 'flex', 'flexWrap': 'wrap'})
])
# callback function to update input container
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown_statistics',component_property='value'))
def update_input_container(statistics_value):
    if statistics_value =='Yearly Statistics': 
        return False
    else: 
        return True

#callback function to update output container to show graph
@app.callback(
    Output('output-container','children'),[Input('select-year','value'),Input('dropdown_statistics','value')])
def update_graph(selected_year, selected_statistic):
    graphs=[]
    if selected_statistic == 'Yearly Statistics':
        # Filter the data for the selected year
        filtered_data = df[df['Year'] == selected_year]

        #line plot for yearly sales for the whole period
        yas= df.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig1 = px.line(yas, x='Year', y='Automobile_Sales', title=" total Automobile Sales ")
        graphs.append(dcc.Graph(figure=fig1))


        #monthly automobile sales
        mas= filtered_data.groupby('Month')['Automobile_Sales'].mean().reset_index()
        fig2 = px.line(mas, x='Month', y='Automobile_Sales', title=" total monthly Automobile Sales ")
        graphs.append(dcc.Graph(figure=fig2))

        #bar chart for averag number fo vehicle sold during the given year
        avg_sales= filtered_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig3 = px.bar(avg_sales, x='Vehicle_Type', y='Automobile_Sales', title=" avg Automobile Sales for each vehcile type")
        graphs.append(dcc.Graph(figure=fig3))

        #pie chart 
        avg_exp= filtered_data.groupby('Vehicle_Type')['Advertising_Expenditure'].mean().reset_index()
        fig4 = px.pie(avg_exp, names='Vehicle_Type', values='Advertising_Expenditure', title=" Average expenditure for each vehcile  type ")
        graphs.append(dcc.Graph(figure=fig4))




        return graphs
        
        

        
        # Create a graph using Plotly Express
        #fig = px.line(filtered_data, x='Month', y='Automobile_Sales', title=f"Automobile Sales in {selected_year}")
        
        #return dcc.Graph(figure=fig)
    
    elif selected_statistic == 'Recession Period Statistics':
        # Filter the data for recession periods (assuming a 'Recession' column exists)
        recession_data = df[df['Recession'] == 1]
        
        # automobile sale fluctuation over recession period
        yas= recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig5 = px.line(yas, x='Year', y='Automobile_Sales', title=" Average Automobile Sales fluctuation over Recession Period ")
        graphs.append(dcc.Graph(figure=fig5))


        #average number of vehcile sold by vehicle type
        avg_sales= recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig6 = px.bar(avg_sales, x='Vehicle_Type', y='Automobile_Sales', title=" Average Automobile Sales by vehicle type ")
        graphs.append(dcc.Graph(figure=fig6))

        #pie chart for total expenditure share by vehicle type
        avg_sales= recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig7 = px.pie(avg_sales, names='Vehicle_Type', values='Automobile_Sales', title=" Average Automobile Sales by vehicle type ")
        graphs.append(dcc.Graph(figure=fig7))

        #bar chart for the relation between unemployment rate and sales for each vehicle type
        unemp_data= recession_data.groupby(['Vehicle_Type','unemployment_rate'])['Automobile_Sales'].mean().reset_index()
        fig8 =px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type', 
             title="Effect of Unemployment Rate on Vehicle Type and Sales",
             labels={'unemployment_rate': 'Unemployment Rate (%)', 'Automobile_Sales': 'Average Sales'},
             barmode='group')
        graphs.append(dcc.Graph(figure=fig8))
        
        return graphs

        
if __name__ == '__main__':
    app.run_server(debug=True,port=8080)


In [142]:
recession_data = df[df['Recession'] == 1]
unemp_data= recession_data.groupby(['Vehicle_Type','unemployment_rate'])['Automobile_Sales'].mean().reset_index()
fig7 =px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type', 
             title="Effect of Unemployment Rate on Vehicle Type and Sales",
             labels={'unemployment_rate': 'Unemployment Rate (%)', 'Automobile_Sales': 'Average Sales'},
             barmode='group')
fig7.show()
#recession_data


avag_sales= filtered_data.groupby('Year')['Automobile_Sales'].mean()
        fig3 = px.bar(avag_sales, x='Year', y='Automobile_Sales', title=" avg Automobile Sales ")
    fig3

In [85]:
filtered_data=df[df['Year']==2020]
yearly_rec=filtered_data.reset_index()
print(yearly_rec.head())

   index        Date  Year Month  Recession  Consumer_Confidence  \
0    109   9/30/2020  2020   Sep          1               111.62   
1    110  10/31/2020  2020   Oct          1                84.48   
2    111  11/30/2020  2020   Nov          1               112.39   
3    112  12/31/2020  2020   Dec          1                97.85   
4    484   1/31/2020  2020   Jan          0               106.81   

   Seasonality_Weight      Price  Advertising_Expenditure  Competition  \
0                0.07  33435.708                     2622            8   
1                0.00  29408.199                     4700            4   
2                0.07  24960.137                     4945            8   
3                0.25  32399.721                     1341            4   
4                0.50  24360.412                     3981            7   

      GDP  Growth_Rate  unemployment_rate  Automobile_Sales     Vehicle_Type  \
0  17.046    -1.170773                3.7             692.8  Mediu